# Libraries

In [ ]:
from sklearn.model_selection import (
    train_test_split, 
    cross_val_score,
    KFold
)
from optuna.visualization import (
    plot_optimization_history, 
    plot_param_importances, 
    plot_slice
)
from sklearn import set_config
#from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from tqdm import tqdm
from openfe import OpenFE, transform, tree_to_formula, ForwardFeatureSelector, TwoStageFeatureSelector

import matplotlib.pyplot as plt
import lightgbm as lgb
import xgboost as xgb
import pandas as pd
import numpy as np
import seaborn as sns
import optuna
import shap
import warnings
import gc

# Configuration

In [ ]:
# Global configurations for sklearn:
set_config(transform_output="pandas")

# Global configurations for pandas:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 50)
pd.set_option("display.precision", 3)
pd.set_option("display.max_colwidth", None)

# Suppress warnings
warnings.filterwarnings("ignore")

In [ ]:
SEED = 42
TRAIN_SIZE = 100000
DATA_DIR = 'data/'

# Functions

In [ ]:
def memory_optimization(train_data: pd.DataFrame) -> pd.DataFrame:
    """ Reduce memory usage of dataframe by modifying type of each column.

    Argument:
    train_data (pd.DataFrame): The DataFrame to be optimized.
    """
    base_memory = train_data.memory_usage().sum() / 1024**2
    print(f'Memory usage of dataframe is {base_memory:.2f} MB')

    for column in train_data.columns:
        column_type = train_data[column].dtype

        if column_type == 'bool':
            continue

        if column_type != 'category':
            column_value_min = train_data[column].min()
            column_value_max = train_data[column].max()
            if str(column_type)[:3] == 'int':
                if column_value_min > np.iinfo(np.int8).min and column_value_max < np.iinfo(np.int8).max:
                    train_data[column] = train_data[column].astype(np.int8)
                elif column_value_min > np.iinfo(np.int16).min and column_value_max < np.iinfo(np.int16).max:
                    train_data[column] = train_data[column].astype(np.int16)
                elif column_value_min > np.iinfo(np.int32).min and column_value_max < np.iinfo(np.int32).max:
                    train_data[column] = train_data[column].astype(np.int32)
                elif column_value_min > np.iinfo(np.int64).min and column_value_max < np.iinfo(np.int64).max:
                    train_data[column] = train_data[column].astype(np.int64)  
            else:
                if column_value_min > np.finfo(np.float16).min and column_value_max < np.finfo(np.float16).max:
                    train_data[column] = train_data[column].astype(np.float16)
                elif column_value_min > np.finfo(np.float32).min and column_value_max < np.finfo(np.float32).max:
                    train_data[column] = train_data[column].astype(np.float32)
                else:
                    train_data[column] = train_data[column].astype(np.float64)
        else:
            train_data[column] = train_data[column].astype('category')

    optimized_memory = train_data.memory_usage().sum() / 1024**2
    optimized_percentage = 100 * (base_memory - optimized_memory) / base_memory

    print(f'Memory usage after optimization is: {optimized_memory:.2f} MB')
    print(f'Decreased by {optimized_percentage:.1f}%')

    return train_data

# Read data

In [ ]:
# Original data
org_data = pd.read_csv(DATA_DIR + 'flood.csv')

# Competition data
org_train = pd.read_csv(DATA_DIR + 'train.csv')

In [ ]:
# Enrich dataset
org_train.drop('id', axis=1, inplace=True)
df = pd.concat([org_train, org_data])
df.reset_index(drop=True, inplace=True)

df.head()

In [ ]:
%%time
# Optimize memory usage
df = memory_optimization(df)

# EDA

In [ ]:
print(f'Before concat: {org_train.shape}, {org_data.shape}\n')
print(f'After concat: {df.shape}')

del org_train, org_data
gc.collect()

In [ ]:
# View columns stats
df.info()

## Null & Duplicates

In [ ]:
# Check for missing values with a horizontal bar plot
df.isnull().sum().plot(kind='barh')
plt.title('Missing values per column')
plt.show()

In [ ]:
# Check for duplication
print(f'Number of duplicated records: {df.duplicated().sum()}')

## Numerical

In [ ]:
# Select only numerical
numerical_columns = list(df.select_dtypes(include=['float16', 'int8']))

In [ ]:
# Check statistics
df[numerical_columns].describe().round(3)

In [ ]:
df.hist(figsize=(15, 12))
plt.title("Numerical features distribution with Histograms")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(data=df[numerical_columns], orient='h')
plt.title('Numerical Features distribution with Boxplot')
plt.show()

In [ ]:
skew_coefficients = df[numerical_columns].skew()
skew_coefficients

In [ ]:
# Check for inf values
for column in df.columns:
    if np.isinf(df[column]).any():
        print(f"Column '{column}' contains inf values.")

## Relationship

In [ ]:
no_target_df = df.drop('FloodProbability', axis=1)
without_target_corr_matrix = no_target_df.corr() # Default is Pearson

# Create a mask for the upper triangle
mask_up_tri = np.triu(np.ones_like(without_target_corr_matrix, dtype=bool))

plt.figure(figsize=(15, 12))
sns.heatmap(without_target_corr_matrix.round(3), annot=True, mask=mask_up_tri)
plt.title("Correlation matrix with Target variable")
plt.show()

In [ ]:
# Calculate the correlation matrix with target
df.corr().abs()["FloodProbability"].sort_values(ascending=False)

## Target

In [ ]:
sns.kdeplot(data=df, x="FloodProbability")
plt.title("Flood Probability distribution")
plt.show()

# Pre-processing

In [ ]:
features = df.drop('FloodProbability', axis=1)
target = df[['FloodProbability']]

del df
gc.collect()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.1, random_state=SEED, #shuffle=True
)

# Check the shape of the train and test data
print(f"Train data shape: {X_train.shape}, {y_train.shape}")
print(f"Test data shape: {X_test.shape}, {y_test.shape}")

# Feature engineering

In [ ]:
%%time
ofe = OpenFE()

# Train on smaller size for faster speed
X_ofe = X_train[:200000]
y_ofe = y_train[:200000]

# Generate new features
gen_feats = ofe.fit(data=X_ofe,
                    label=y_ofe, 
                    task='regression', 
                    metric='rmse',
                    feature_boosting=True, 
                    n_jobs=6, 
                    verbose=False)

In [ ]:
print(f'Total features generated: {len(ofe.new_features_list)}\n')

# Top 20 features
for feature in ofe.new_features_list[:20]:
    print(tree_to_formula(feature))

In [ ]:
%%time
X_train_sub = X_train[:TRAIN_SIZE]
y_train_sub = y_train[:TRAIN_SIZE]

X_train_sub, X_test = transform(X_train_sub, X_test, gen_feats, n_jobs=6)

In [ ]:
%%time
fs = ForwardFeatureSelector(task='regression', verbose=False)

fs.fit(data=X_train_sub, label=y_train_sub)

In [ ]:
X_train_testfs = fs.transform(X_train_sub)

# Metric

Submissions are evaluated using the R2 score.

# Modeling

## Optuna

In [ ]:
""" %%time
def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'l2',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_data_in_leaf': trial.suggest_int("min_data_in_leaf", 1, 100),
        #'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart']),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'n_jobs': -1,
        'random_state': SEED,
        'verbosity': -1
    }

    # Cross-validation setup
    kfold = KFold(n_splits=10, shuffle=True, random_state=SEED)
    r2_scores = []

    for train_index, valid_index in kfold.split(X_train, y_train):
        train_data = lgb.Dataset(X_train.iloc[train_index], label=y_train.iloc[train_index])
        valid_data = lgb.Dataset(X_train.iloc[valid_index], label=y_train.iloc[valid_index])

        # Train the model
        gbm = lgb.train(
            params,
            train_data,
            num_boost_round=100,
            valid_sets=[train_data, valid_data],
            valid_names=['train', 'valid'],
            callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=False)]
        )

        # Predict and calculate R2
        y_pred = gbm.predict(X_train.iloc[valid_index], num_iteration=gbm.best_iteration)
        r2 = r2_score(y_train.iloc[valid_index], y_pred)
        r2_scores.append(r2)

    return np.mean(r2_scores)


study = optuna.create_study(study_name="LightGBM", direction="maximize")
study.optimize(objective, n_trials=50, n_jobs=-1, show_progress_bar=True)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")
print() """

""" LGBM
Number of finished trials:  50
Best trial:
  Value:  0.8448460037468983
  Params: 
    n_estimators: 823
    learning_rate: 0.042084691701664546
    subsample: 0.8741517323343981
    colsample_bytree: 0.5741858210937979
    min_data_in_leaf: 22
    num_leaves: 39
    bagging_freq: 7

CPU times: user 1d 11h 23min 55s, sys: 4h 16min 26s, total: 1d 15h 40min 21s
Wall time: 6h 57min 11s
"""

In [ ]:
""" %%time
def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        "max_depth": trial.suggest_int("max_depth", 1, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "n_estimators": trial.suggest_int('n_estimators', 100, 1000),
        'n_jobs': -1,
        'random_state': SEED,
    }

    # Cross-validation setup
    kfold = KFold(n_splits=10, shuffle=True, random_state=SEED)
    r2_scores = []

    for train_index, valid_index in kfold.split(X_train, y_train):
        # Create DMatrix for training and validation sets with appropriate weights
        dtrain = xgb.DMatrix(data=X_train.iloc[train_index], label=y_train.iloc[train_index])
        dvalid = xgb.DMatrix(data=X_train.iloc[valid_index], label=y_train.iloc[valid_index])

        # Train the model
        bst = xgb.train(
            params,
            dtrain,
            num_boost_round=100,
            evals=[(dvalid, 'validation')],
            early_stopping_rounds=10,
            verbose_eval=False
        )

        # Predict probabilities for the validation set
        y_pred = bst.predict(dvalid)

        # Calculate R2 and store the score
        r2 = r2_score(y_train.iloc[valid_index], y_pred)
        r2_scores.append(r2)
    
    return np.mean(r2_scores)


study = optuna.create_study(study_name="XGBoost", direction="maximize")
study.optimize(objective, n_trials=50, n_jobs=-1, show_progress_bar=True)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")
print() """

""" XGBoost
{'max_depth': 8,
 'min_child_weight': 20,
 'subsample': 0.07190228850100583,
 'colsample_bytree': 0.5615919991275306,
 'learning_rate': 0.09054610792000152,
 'n_estimators': 505}

Number of finished trials:  50
Best trial:
  Value:  0.7975471022803855
  Params: 
    max_depth: 8
    min_child_weight: 20
    subsample: 0.07190228850100583
    colsample_bytree: 0.5615919991275306
    learning_rate: 0.09054610792000152
    n_estimators: 505

CPU times: user 2h 11min 31s, sys: 22min 16s, total: 2h 33min 47s
Wall time: 35min 13s
"""

In [ ]:
""" %%time
def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        'random_state': SEED,
        'loss_function': 'RMSE',
        'eval_metric': 'R2',
        'early_stopping_rounds': 10,
        'verbose': False
    }

    # Cross-validation setup
    kfold = KFold(n_splits=10, shuffle=True, random_state=SEED)
    r2_scores = []

    for train_index, valid_index in kfold.split(X_train, y_train):
        X_train_split, X_valid_split = X_train.iloc[train_index], X_train.iloc[valid_index]
        y_train_split, y_valid_split = y_train.iloc[train_index], y_train.iloc[valid_index]

        model = CatBoostRegressor(**params)
        model.fit(X_train_split, y_train_split,
                  eval_set=(X_valid_split, y_valid_split),
                  use_best_model=True)

        # Predict probabilities for the validation set
        y_pred = model.predict(X_valid_split)

        # Calculate R2 and store the score
        r2 = r2_score(y_valid_split, y_pred)
        r2_scores.append(r2)

    return np.mean(r2_scores)


study = optuna.create_study(study_name="CatBoost", direction="maximize")
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")
print() """

""" CatBoost
{'iterations': 991,
 'learning_rate': 0.09877620412322519,
 'depth': 5,
 'subsample': 0.9609189454297099,
 'colsample_bylevel': 0.8196165977934415,
 'min_data_in_leaf': 81}

Number of finished trials:  50
Best trial:
  Value:  0.8497594589181798
  Params: 
    iterations: 991
    learning_rate: 0.09877620412322519
    depth: 5
    subsample: 0.9609189454297099
    colsample_bylevel: 0.8196165977934415
    min_data_in_leaf: 81

CPU times: user 15h 56min 41s, sys: 43min 58s, total: 16h 40min 39s
Wall time: 4h 14min 36s
"""

In [ ]:
xgb = XGBRegressor(
    max_depth=8,
    min_child_weight=20,
    subsample=0.07190228850100583,
    colsample_bytree=0.5615919991275306,
    learning_rate=0.09054610792000152,
    n_estimators=505,
    objective='reg:squarederror',
    # eval_metric='auc',
    # tree_method='gpu_hist',  # Uncomment if using GPU
    # random_state=SEED,
    n_jobs=-1
)

lgbm = LGBMRegressor(
    n_estimators=823,
    learning_rate=0.042084691701664546,
    subsample=0.8741517323343981,
    colsample_bytree=0.5741858210937979,
    min_data_in_leaf=22,
    boosting_type='gbdt',
    num_leaves=39,
    bagging_freq=7,
    objective='regression',
    metric='l2',
    # device_type='gpu', 
    # random_state=SEED,
    verbosity=-1,
    num_threads=-1
)

cat = CatBoostRegressor(
    iterations=991,
    learning_rate=0.09877620412322519,
    depth=5,
    subsample=0.9609189454297099,
    colsample_bylevel=0.8196165977934415,
    min_data_in_leaf=81,
    random_state=SEED,
    loss_function='RMSE',
    eval_metric='R2',
    early_stopping_rounds=10,
    verbose=False
)

## Original features

In [ ]:
%%time
# Cross-validation split
kfold = KFold(n_splits=10, shuffle=True, random_state=SEED)

xgb_scores = cross_val_score(xgb, X_train, y_train, cv=kfold, scoring='r2', n_jobs=-1)
lgbm_scores = cross_val_score(lgbm, X_train, y_train, cv=kfold, scoring='r2', n_jobs=-1)
cat_scores = cross_val_score(cat, X_train, y_train, cv=kfold, scoring='r2', n_jobs=-1)

cv_scores = [xgb_scores, lgbm_scores, cat_scores]

r2_df = pd.DataFrame(
    {
        'Model': ['XGBoost', 'LightGBM', 'CatBoost'],
        'Mean R2': np.mean(cv_scores, axis=1)
    }
)
r2_df

## OpenFE features

In [ ]:
%%time
# Cross-validation setup
kfold = KFold(n_splits=10, shuffle=True, random_state=SEED)

r2_scores = []
for train_index, valid_index in tqdm(kfold.split(X_train_sub, y_train_sub), total=kfold.get_n_splits(), desc='K-Fold Progress'):
    train_data = lgb.Dataset(X_train_sub.iloc[train_index], label=y_train_sub.iloc[train_index])
    valid_data = lgb.Dataset(X_train_sub.iloc[valid_index], label=y_train_sub.iloc[valid_index])

    params = {
        'n_estimators': 823,
        'learning_rate': 0.042084691701664546,
        'subsample': 0.8741517323343981,
        'colsample_bytree': 0.5741858210937979,
        'min_data_in_leaf': 22,
        'boosting_type': 'gbdt',
        'num_leaves': 39,
        'bagging_freq': 7,
        'objective': 'regression',
        'metric': 'l2',
        #'device_type': 'gpu',  
        'random_state': SEED,  
        'verbosity': -1,
        'num_threads': -1  # Use all available threads
    }

    # Train the model
    gbm = lgb.train(
        params,
        train_data,
        num_boost_round=100,
        valid_sets=[train_data, valid_data],
        valid_names=['train', 'valid'],
        callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=False)]
    )

    # Predict and calculate R2
    y_pred = gbm.predict(X_train_sub.iloc[valid_index], num_iteration=gbm.best_iteration)
    r2 = r2_score(y_train_sub.iloc[valid_index], y_pred)
    r2_scores.append(r2)

print(f'Mean R2: {np.mean(r2_scores):.3f}')

In [ ]:
%%time
# Cross-validation setup
kfold = KFold(n_splits=10, shuffle=True, random_state=SEED)

r2_scores = []
for train_index, valid_index in tqdm(kfold.split(X_train_sub, y_train_sub), total=kfold.get_n_splits(), desc='K-Fold Progress'):
    # Create DMatrix for training and validation sets with appropriate weights
    dtrain = xgb.DMatrix(data=X_train_sub.iloc[train_index], label=y_train_sub.iloc[train_index], enable_categorical=True)
    dvalid = xgb.DMatrix(data=X_train_sub.iloc[valid_index], label=y_train_sub.iloc[valid_index], enable_categorical=True)

    params = {
        'max_depth': 8,
        'min_child_weight': 20,
        'subsample': 0.07190228850100583,
        'colsample_bytree': 0.5615919991275306,
        'learning_rate': 0.09054610792000152,
        'n_estimators': 505,
        'objective': 'reg:squarederror',
        'random_state': SEED,  
        'n_jobs': -1,  # Use all available threads
        'enable_categorical': True
    }

    # Train the model
    bst = xgb.train(
        params,
        dtrain,
        num_boost_round=100,
        evals=[(dvalid, 'validation')],
        early_stopping_rounds=10,
        verbose_eval=False
    )

    # Predict probabilities for the validation set
    y_pred = bst.predict(dvalid)

    # Calculate R2 and store the score
    r2 = r2_score(y_train_sub.iloc[valid_index], y_pred)
    r2_scores.append(r2)
    
print(f'Mean R2: {np.mean(r2_scores):.3f}')

In [ ]:
%%time
categorical_features = X_train_sub.select_dtypes(include=['category']).columns.tolist()
for column in categorical_features:
    X_train_sub[column] = X_train_sub[column].astype(str)

kfold = KFold(n_splits=10, shuffle=True, random_state=SEED)

r2_scores = []
for train_index, valid_index in tqdm(kfold.split(X_train_sub, y_train_sub), total=kfold.get_n_splits(), desc='K-Fold Progress'):
    X_train_split, X_valid_split = X_train_sub.iloc[train_index], X_train_sub.iloc[valid_index]
    y_train_split, y_valid_split = y_train_sub.iloc[train_index], y_train_sub.iloc[valid_index]

    model = CatBoostRegressor(
            iterations=991,
            learning_rate=0.09877620412322519,
            depth=5,
            subsample=0.9609189454297099,
            colsample_bylevel=0.8196165977934415,
            min_data_in_leaf=81,
            random_state=SEED,
            loss_function='RMSE',
            eval_metric='R2',
            early_stopping_rounds=10,
            verbose=False,
            cat_features=categorical_features
        )
    model.fit(X_train_split, y_train_split,
                eval_set=(X_valid_split, y_valid_split),
                use_best_model=True)

    # Predict probabilities for the validation set
    y_pred = model.predict(X_valid_split)

    # Calculate R2 and store the score
    r2 = r2_score(y_valid_split, y_pred)
    r2_scores.append(r2)

print(f'Mean R2: {np.mean(r2_scores):.3f}')

## Selected features

# Evaluation

In [ ]:
cat.fit(X_train, y_train)
y_pred_cat = cat.predict(X_test)
r2_cat = r2_score(y_test, y_pred_cat)
print(f'CatBoost R2: {r2_cat}')

In [ ]:
categorical_features = X_train_testfs.select_dtypes(include=['category']).columns.tolist()
for column in categorical_features:
    X_train_testfs[column] = X_train_testfs[column].astype(str)

best_model = CatBoostRegressor(
            iterations=991,
            learning_rate=0.09877620412322519,
            depth=5,
            subsample=0.9609189454297099,
            colsample_bylevel=0.8196165977934415,
            min_data_in_leaf=81,
            random_state=SEED,
            loss_function='RMSE',
            eval_metric='R2',
            early_stopping_rounds=10,
            verbose=False,
            cat_features=categorical_features
        )

In [ ]:
best_model.fit(X_train_testfs, y_train_sub)

X_test = fs.transform(X_test)
for column in categorical_features:
    X_test[column] = X_test[column].astype(str)
    
y_pred_cat = best_model.predict(X_test)
r2_cat = r2_score(y_test, y_pred_cat)
print(f'CatBoost R2: {r2_cat}')

# Prediction & Submission

In [ ]:
# Retrain before prediction
cat.fit(features, target)

# Predict the target variable
test = pd.read_csv(DATA_DIR + 'test.csv')
predictions = cat.predict(test.drop('id', axis=1))

In [ ]:
%%time
test_df = pd.DataFrame()
test = pd.read_csv(DATA_DIR + 'test.csv')
test.drop('id', axis=1, inplace=True)
test, test_df = transform(test, test_df, new_features_list=gen_feats, n_jobs=6)

In [ ]:
test = fs.transform(test)
predictions = best_model.predict(test)

In [ ]:
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')
sub['FloodProbability'] = predictions
sub.to_csv('submission.csv', index=False)

# Test Autogluon

In [ ]:
""" 0.85831 
label = "FloodProbability"
predictor = TabularPredictor(label=label).fit(df)

test = pd.read_csv(DATA_DIR + 'test.csv')
predictions = predictor.predict(test.drop('id', axis=1))

sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')
sub[label] = predictions
sub.to_csv('submission.csv', index=False)
"""

# Reference